Задание:
> Deliver transactions from one database to another
> 
> Maintain consistency in the face of every component outage
> 
> Use queue or implement direct outbox-to-inbox

Я поднял два Постгреса из docker-compose и Кафку. Кафка сконфигурирована так же как в предыдущих заданиях. В постгресах создал такие таблицы:
* pg_start 8000:5432
    * main.transactions
    * outbox.transactions
    * main.balance
* pg_end 8001:5432
    * main.transactions
    * main.balance

Задача в том, чтобы вручную реплицировать данные из `main.transactions`. Балансы в `main.balance` должны по итогу совпадать, так как они вычисляются на основе `main.transactions`. Каждая транзакция должна быть лишь единожды учитана в балансах, какие бы ошибки не происходили.

В классе PostgreSQL у меня реализована кастомная обёртка к Постгресу на основе asyncpg.

In [1]:
from PostgreSQL import PostgreSQL
pg_start = await PostgreSQL(user='postgres', password='postgres', database='db_start', host='localhost', port=8000).connect()
pg_end = await PostgreSQL(user='postgres', password='postgres', database='db_end', host='localhost', port=8001).connect()

await pg_start.execute('TRUNCATE TABLE main.transactions')
await pg_start.execute('TRUNCATE TABLE outbox.transactions')
await pg_start.execute('TRUNCATE TABLE main.balance')
await pg_end.execute('TRUNCATE TABLE main.transactions')
await pg_end.execute('TRUNCATE TABLE main.balance')

display(await pg_start.select("SELECT 'pg_start is ready' AS status"))
display(await pg_end.select("SELECT 'pg_end is ready' AS status"))

,status
0,pg_start is ready


,status
0,pg_end is ready


Огранизую запись данных в pg_start. 
Буду пользоваться декоратором `use_transaction`, который передаёт внутрь декорированной функции коннект с открытой транзакцией.

In [ ]:
import asyncio
import random
import asyncpg
from uuid import uuid4
from datetime import datetime

def use_transaction(postgres_wrapper):
    def _use_transaction(func):
        async def _use_transaction_aync(*args, **kwargs):
            try:
                async with postgres_wrapper.transaction() as connection:
                    await func(
                        pg=postgres_wrapper, 
                        connection=connection,
                        *args, **kwargs)
            except asyncpg.UniqueViolationError:
                print('Faced Unique Violation Error. Skipping')
        return _use_transaction_aync
    return _use_transaction


@use_transaction(pg_start)
async def generate_new_row(pg, connection, transaction_id=None):
    round_to_ten = lambda x: x - (x % 10)
    payload = [
        transaction_id or hash(uuid4()), # id
        random.randint(1, 10),           # account_id
        round_to_ten(random.randint(10, 100)) * random.choice([1, -1]),  # amount
        datetime.now()]                  # created_at
    await pg.insert('INSERT INTO main.transactions (id, account_id, amount, created_at) VALUES', payload, con=connection)
    await pg.insert('INSERT INTO outbox.transactions (id, account_id, amount, created_at) VALUES', payload, con=connection)
    await pg.insert("""
        INSERT INTO main.balance (account_id, current_balance, updated_at)
        VALUES
        ON CONFLICT (account_id) DO
            UPDATE SET current_balance = balance.current_balance + EXCLUDED.current_balance, updated_at = EXCLUDED.updated_at
            RETURNING current_balance""", 
        [payload[1], payload[2], payload[3]], 
        con=connection)

for x in range(5):
    await generate_new_row()
df = await pg_start.select('SELECT * FROM main.transactions')
display(df)
display(await pg_start.select('SELECT * FROM outbox.transactions'))
display(await pg_start.select('SELECT * FROM main.balance'))

,id,account_id,amount,created_at
0,1472072969025220725,5,50.00,2025-07-23 20:06:21.085330
1,451141627490821775,1,60.00,2025-07-23 20:06:21.090884
2,1475064741160474357,10,90.00,2025-07-23 20:06:21.093729
3,1043847060973054025,1,-80.00,2025-07-23 20:06:21.096271
4,1216133869908735481,2,90.00,2025-07-23 20:06:21.098962


,id,account_id,amount,created_at
0,1472072969025220725,5,50.00,2025-07-23 20:06:21.085330
1,451141627490821775,1,60.00,2025-07-23 20:06:21.090884
2,1475064741160474357,10,90.00,2025-07-23 20:06:21.093729
3,1043847060973054025,1,-80.00,2025-07-23 20:06:21.096271
4,1216133869908735481,2,90.00,2025-07-23 20:06:21.098962


,account_id,current_balance,updated_at
0,5,50.00,2025-07-23 20:06:21.085330
1,10,90.00,2025-07-23 20:06:21.093729
2,1,-20.00,2025-07-23 20:06:21.096271
3,2,90.00,2025-07-23 20:06:21.098962


Я не стал отделять сервис, который вставляет в базу от сервиса, который генерирует сообщения. Если бы они были отделены, от сервиса-генератора сообщений требовалась бы идемпотентная генерация поля `id` - идентификатора транзации.

По идентификатору транзакции в `main.transactions` создан первичный ключ, так что если сервис попытается повторно записать строку к совпадающим `id`, Постгрес не позволит этого сделать.

In [3]:
await generate_new_row(transaction_id=df.iloc[0, 0])
display(await pg_start.select('SELECT * FROM main.balance'))

Faced Unique Violation Error. Skipping


,account_id,current_balance,updated_at
0,5,50.00,2025-07-23 20:06:21.085330
1,10,90.00,2025-07-23 20:06:21.093729
2,1,-20.00,2025-07-23 20:06:21.096271
3,2,90.00,2025-07-23 20:06:21.098962


Пока мы не получаем подтверждения об успешном выполнения каждой из трёх команд к базе, то не закрываем транзакцию. А без закрытой транзакции не будет доведена до конца ни одна из команд. Так что либо вставка успешно выполняется во все три таблицы, либо ни в одну. 
По этой же причине в таблице в outbox можно не создавать CONSTRAINT-ы - все необходимые проверки будут происходить при вставке в основную таблицу.

Теперь из outbox перенесу данные в Kafka. Пока не очень понимаю, для чего здесь нужна Kafka, если можно писать в целевую базу напрямую, но интересно попробовать. 

In [4]:
!../task_3/kafka/bin/kafka-topics.sh --delete --topic outbox-transactions --bootstrap-server localhost:19092,localhost:19091,localhost:19090

In [5]:
!../task_3/kafka/bin/kafka-topics.sh --create --topic outbox-transactions --partitions 1 --bootstrap-server localhost:19092,localhost:19091,localhost:19090

Created topic outbox-transactions.


In [ ]:
import json
from confluent_kafka import Producer

producer_config = {
    'bootstrap.servers': 'localhost:19090,localhost:19091,localhost:19092',
    'enable.idempotence': True,
    'acks': 'all',
    'retries': 5,
    'batch.size': 65536,
    'linger.ms': 1000}
producer = Producer(producer_config)

def delivery_report(err, msg):
    if err is not None:
        print(f'Message delivery failed: {err}')
    else:
        print(f'Delivered {msg.value().decode()}')

@use_transaction(pg_start)
async def transfer_outbox_to_kafka(pg, connection):
    for _id, account_id, amount, created_at in await pg_start.select("""
            DELETE FROM outbox.transactions
            WHERE ctid IN (SELECT ctid FROM outbox.transactions LIMIT 100)
            RETURNING id, account_id, amount, created_at
        """, con=connection).values:
        message = dict(id=_id, account_id=account_id, amount=int(amount), created_at=str(created_at))
        producer.produce(
            'outbox-transactions',
            key=str(_id),
            value=json.dumps(message),
            callback=delivery_report)
    producer.flush()

Продьюсер предпримет пять попыток отправить сообщение в Кафку так, чтобы запись прошла на минимально допустимое количество in-sync реплик. Благодаря `enable.idempotence=true` мы имеем гарантию, что в результате этих пяти попыток сообщение не запишется больше одного раза. 

Одна строка из таблицы будет весить примерно 160 байт. Если мы ставим размер батча в 65536 байт, не следует отправлять больше, чем 300 строк за раз, иначе продьюсер предпримет несколько попыток отправить сообщение, и первая попытка можешь быть успешной, а вторая - нет. Тогда мы не узнаем о том, что часть данных до Кафки успешно дошла. 

Если возникает ошибка до записи в Кафку, то транзакция обрывается, и строки остаются в `outbox.transactions`. Но если ошибка возникает после успешной записи в Кафку (после `producer.flush()`, например), запись тех же данных повторится ещё раз, так как транзакция с `DELETE`-ом из `outbox.transactions` всё-равно оборвётся. В принципе, это не страшно, потому что принимающей стороной будет Постгрес с первичным ключом по полю `id`, который не допустит дублей. Но попробуем и на этом этапе приблизиться к exactly-once.

Пусть декоратор `use_transaction` дополнительно принимает путь к файлу. В этот recovery-файл будет сохранять вывод декорированной функции - список идентификаторов, запушенных в Кафку - но только в том случае, если:
* уже получил вывод от функции, записывающей в Кафку (значит, Кафка подтвердила успешную запись)
* сталкивается с ошибкой при закрытии транзакции (потому что нет других мест, где он может споткнуться)

Запись в файловую систему не отказывает так часто, как сеть, так что будем считать, что на этапе сохранения в файлик ошибок быть не может. Самое сложное, видимо, будет таскать recovery-файлик вслед за outbox-приложением, если последнее будет развёрнуто в кубере/облаке и будет переодически менять хоста.

In [ ]:
def use_transaction_with_recovery_file(postgres_wrapper, recovery_path=None):
    def _use_transaction(func):
        async def _use_transaction_aync(*args, **kwargs):
            with open(recovery_path, 'r') as file:
                already_processed_ids = set(map(int, filter(lambda x: x, file.read().split(' '))))
            func_output = None
            if recovery_path is not None:
                kwargs['already_processed_ids'] = already_processed_ids
            try:
                async with postgres_wrapper.transaction() as connection:
                    func_output = await func(
                        pg=postgres_wrapper, 
                        connection=connection,
                        *[x for x in args if x != 'error'], 
                        **kwargs)
                    if 'error' in args:
                        raise Exception('raised intentionally')
                    
            except asyncpg.UniqueViolationError:
                print('Faced Unique Violation Error. Skipping')
                
            except Exception as exc:
                if recovery_path is not None and func_output is not None:
                    # если запись успешна, но транзакция не закрылась, то добавляю записанные идентификаторы в файл
                    with open(recovery_path, 'w') as file:
                        file.write(' '.join(map(str, already_processed_ids.union(func_output))) + ' ')
                raise exc
            
            else:
                ids_that_should_remain = already_processed_ids.difference(func_output)
                if len(already_processed_ids) > 0 and len(ids_that_should_remain) != len(already_processed_ids):
                    # если запись в Кафку успешна и если строки удалены из таблицы в схеме outbox, то удаляю их из файла
                    with open(recovery_path, 'w') as file:
                        file.write(' '.join(map(str, ids_that_should_remain)) + ' ')
        return _use_transaction_aync
    return _use_transaction


RECOVERY_PATH = '/home/timosha/wb/queues-devhands/task_4/recovery.txt'
@use_transaction_with_recovery_file(pg_start, recovery_path=RECOVERY_PATH)
async def transfer_outbox_to_kafka(pg, connection, already_processed_ids): 
    new_ids = []
    for _id, account_id, amount, created_at in (await pg.select("""
            DELETE FROM outbox.transactions
            WHERE ctid IN (SELECT ctid FROM outbox.transactions LIMIT 100)
            RETURNING id, account_id, amount, created_at""", con=connection)).values:
        
        new_ids.append(_id)
        if _id in already_processed_ids:
            continue
        message = dict(id=_id, account_id=account_id, amount=int(amount), created_at=str(created_at))
        producer.produce(
            'outbox-transactions',
            key=str(_id),
            value=json.dumps(message),
            callback=delivery_report)
    producer.flush()
    return new_ids

with open(RECOVERY_PATH, 'w') as file:
    file.write('')

In [9]:
try:
    await transfer_outbox_to_kafka('error')
except Exception as exc:
    print('\nException: ' + str(exc))

Delivered {"id": 1472072969025220725, "account_id": 5, "amount": 50, "created_at": "2025-07-23 20:06:21.085330"}
Delivered {"id": 451141627490821775, "account_id": 1, "amount": 60, "created_at": "2025-07-23 20:06:21.090884"}
Delivered {"id": 1475064741160474357, "account_id": 10, "amount": 90, "created_at": "2025-07-23 20:06:21.093729"}
Delivered {"id": 1043847060973054025, "account_id": 1, "amount": -80, "created_at": "2025-07-23 20:06:21.096271"}
Delivered {"id": 1216133869908735481, "account_id": 2, "amount": 90, "created_at": "2025-07-23 20:06:21.098962"}

Exception: raised intentionally


In [ ]:
from confluent_kafka import Consumer

def get_msg_from_topic():
    consumer_config = {
        'bootstrap.servers': 'localhost:19090,localhost:19091,localhost:19092',
        'group.id': str(uuid4()),
        'auto.offset.reset': 'earliest'}
    consumer = Consumer(consumer_config)
    consumer.subscribe(['outbox-transactions'])
    for _ in range(10):
        msg = consumer.poll(timeout=1.0)
        if msg is None:
            continue
        print(msg.value())
        
get_msg_from_topic()

b'{"id": 1472072969025220725, "account_id": 5, "amount": 50, "created_at": "2025-07-23 20:06:21.085330"}'
b'{"id": 451141627490821775, "account_id": 1, "amount": 60, "created_at": "2025-07-23 20:06:21.090884"}'
b'{"id": 1475064741160474357, "account_id": 10, "amount": 90, "created_at": "2025-07-23 20:06:21.093729"}'
b'{"id": 1043847060973054025, "account_id": 1, "amount": -80, "created_at": "2025-07-23 20:06:21.096271"}'
b'{"id": 1216133869908735481, "account_id": 2, "amount": 90, "created_at": "2025-07-23 20:06:21.098962"}'


In [11]:
await pg_start.select('SELECT * FROM outbox.transactions')

,id,account_id,amount,created_at
0,1472072969025220725,5,50.00,2025-07-23 20:06:21.085330
1,451141627490821775,1,60.00,2025-07-23 20:06:21.090884
2,1475064741160474357,10,90.00,2025-07-23 20:06:21.093729
3,1043847060973054025,1,-80.00,2025-07-23 20:06:21.096271
4,1216133869908735481,2,90.00,2025-07-23 20:06:21.098962


Вот пример отказа после коммита в Кафку: строки есть в Кафке, но не удалены из outbox. 

In [12]:
with open(RECOVERY_PATH, 'r') as file:
    print(file.read())

1043847060973054025 451141627490821775 1475064741160474357 1472072969025220725 1216133869908735481 


Идентификаторы транзакций записаны в recovery-файл, и на следующий запуск не будем их отправлять в Кафку, но удалим из outbox. Если удалятся успешно, то сотрём их из recovery-файлика.

In [13]:
await transfer_outbox_to_kafka()
get_msg_from_topic()

b'{"id": 1472072969025220725, "account_id": 5, "amount": 50, "created_at": "2025-07-23 20:06:21.085330"}'
b'{"id": 451141627490821775, "account_id": 1, "amount": 60, "created_at": "2025-07-23 20:06:21.090884"}'
b'{"id": 1475064741160474357, "account_id": 10, "amount": 90, "created_at": "2025-07-23 20:06:21.093729"}'
b'{"id": 1043847060973054025, "account_id": 1, "amount": -80, "created_at": "2025-07-23 20:06:21.096271"}'
b'{"id": 1216133869908735481, "account_id": 2, "amount": 90, "created_at": "2025-07-23 20:06:21.098962"}'


In [14]:
await pg_start.select('SELECT * FROM outbox.transactions')

,id,account_id,amount,created_at


In [15]:
with open(RECOVERY_PATH, 'r') as file:
    print(file.read())

Всё работает так, как предполагалось. Осталось написать inbox-приёмник из Кафки.

In [ ]:
async def transfer_kafka_to_inbox():
    consumer_config = {
        'bootstrap.servers': 'localhost:19090,localhost:19091,localhost:19092',
        'group.id': str(uuid4()),
        'enable.auto.commit': False,
        'auto.offset.reset': 'earliest'}
    consumer = Consumer(consumer_config)
    consumer.subscribe(['outbox-transactions'])
    for _ in range(10):  # или while True
        msg = consumer.poll(timeout=1.0)
        if msg is None:
            await asyncio.sleep(0.5)
            continue
        try:
            message = json.loads(msg.value())
            message['created_at'] = datetime.strptime(message['created_at'], '%Y-%m-%d %H:%M:%S.%f')
            await write_to_db_end(payload=list(message.values()))
            consumer.commit(message=msg)
        except asyncpg.UniqueViolationError:
            print('Faced Unique Violation Error. Skipping')
            consumer.commit(message=msg)
        except Exception as exc:
            print(str(exc))
            await asyncio.sleep(1)

@use_transaction(pg_end)
async def write_to_db_end(pg, connection, payload):
    await pg.insert('INSERT INTO main.transactions (id, account_id, amount, created_at) VALUES', payload, con=connection)
    await pg.insert("""
        INSERT INTO main.balance (account_id, current_balance, updated_at)
        VALUES
        ON CONFLICT (account_id) DO
            UPDATE SET current_balance = balance.current_balance + EXCLUDED.current_balance, updated_at = EXCLUDED.updated_at
            RETURNING current_balance""", 
        [payload[1], payload[2], payload[3]], 
        con=connection)

await transfer_kafka_to_inbox()

`enable.auto.commit=false` даёт гарантию, что если во время обработки случится ошибка и приложение упадёт раньше, чем закоммитит прочтение, оно получит сообщение снова. Это нормально. Худший сценарий - мы попытаемся вставить сообщение повторно. Но в этом случае споткнёмся об UNIQUE CONSTRAINT. 

In [17]:
display(await pg_end.select('SELECT * FROM main.transactions'))
display(await pg_end.select('SELECT * FROM main.balance'))

,id,account_id,amount,created_at
0,1472072969025220725,5,50.00,2025-07-23 20:06:21.085330
1,451141627490821775,1,60.00,2025-07-23 20:06:21.090884
2,1475064741160474357,10,90.00,2025-07-23 20:06:21.093729
3,1043847060973054025,1,-80.00,2025-07-23 20:06:21.096271
4,1216133869908735481,2,90.00,2025-07-23 20:06:21.098962


,account_id,current_balance,updated_at
0,5,50.00,2025-07-23 20:06:21.085330
1,10,90.00,2025-07-23 20:06:21.093729
2,1,-20.00,2025-07-23 20:06:21.096271
3,2,90.00,2025-07-23 20:06:21.098962


На этом, думаю, можно закончить!